In [26]:

import math
import torch
import torchvision
import torchvision.transforms as transforms
import torch.utils
import torch.utils.data
import torch.nn as nn
import torchsummary
import torch.nn.functional as F
import numpy as np
import random

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor()
        ,transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
     ])
train=torchvision.datasets.CIFAR10("/home/spometun/datasets/research", train=True,
                                  download=True, transform=transform)
test=torchvision.datasets.CIFAR10("/home/spometun/datasets/research", train=False,
                                  download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [2]:
def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

set_seed(2)
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
# torchsummary.summary(net, (3, 32, 32), device=torch.device("cpu"))

In [42]:
f = nn.CrossEntropyLoss()
logits = torch.Tensor([[0, 1.0]])
target = torch.tensor([1])
res = f(logits, target)
print(res)
softmax = torch.nn.Softmax(dim=1)(logits)
print(softmax[0])
s = softmax[0]
0 * math.log(1/s[0]) + math.log(1/s[1])
# math.exp(logits[0][0]) / (torch.sum(torch.exp(logits)))
1/math.log(2)

tensor(0.3133)
tensor([0.2689, 0.7311])


1.4426950408889634

In [44]:
train_loader = torch.utils.data.DataLoader(train, batch_size=4, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=4, shuffle=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 1):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 0:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
            pred = net(train[0])
            labels = train[1]
            pass

print('Finished Training')

[1,  2000] loss: 1.939


KeyboardInterrupt: 